# Import the necessary packages

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import re
from scipy.spatial.transform import Rotation as R

# Imread the csv information

In [ ]:
def read_csv_file(file_path):
    servoing_informations = []
    counter = 0
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # skip header

        for row in csv_reader:
            if counter == 0:
                goal_informations = row
            else:
                servoing_informations.append(row)
            counter += 1
    return goal_informations, servoing_informations

In [ ]:
goal_informations, servoing_informations= read_csv_file("/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/keypoints_servoing.csv")
#goal_informations, servoing_informations= read_csv_file("/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/keypoints_servoing_hough.csv")
#goal_informations, servoing_informations= read_csv_file("/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/keypoints_servoing_constantin_uber_Stecker.csv")
#goal_informations, servoing_informations= read_csv_file("/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/keypoints_servoing_constantin_5mm_v2.csv")

# Get the time values

In [ ]:
interference_neural_networks = [] # only nn
time_for_keypoint_extraction = [] # keypoint extraction function
time_for_servoing_loop = [] # hole servoing loop
time_for_picture = [] # time for taking a picture

for i in range(len(servoing_informations)):
    time_for_servoing_loop.append(float(servoing_informations[i][2]))
    interference_neural_networks.append(float(servoing_informations[i][3]))
    time_for_keypoint_extraction.append(float(servoing_informations[i][4]))
    time_for_picture.append(float(servoing_informations[i][6]))


In [ ]:
# Generate random data for mean values (example)
mean_interference_neural_networks = np.array(interference_neural_networks).mean()
mean_time_for_keypoint_extraction = np.array(time_for_keypoint_extraction).mean()
mean_time_for_servoing_loop = np.array(time_for_servoing_loop).mean()
mean_time_for_picture = np.array(time_for_picture).mean()

print("Mean Interference Neural Network: ", mean_interference_neural_networks)
print("Mean Time for Interference + Keypoint Extraction: ", mean_time_for_keypoint_extraction)
print("Mean Time for Servoing Loop: ", mean_time_for_servoing_loop)
print("Mean Time for Taking a Picture: ", mean_time_for_picture)


# Create the figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

# Plot the upper code on the left subplot (ax1)
ax1.plot(interference_neural_networks, label="Interference Neural Network")
ax1.plot(time_for_keypoint_extraction, label="Time for Interference + Keypoint Extraction")
ax1.plot(time_for_servoing_loop, label="Time for Servoing Loop")
ax1.plot(time_for_picture, label="Time for Taking a Picture")
ax1.set_xlabel("Servoing Step", fontsize=14)
ax1.set_ylabel("Time in s", fontsize=14)
legend = ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2, fontsize=14)
ax1.set_ylim(0, max(mean_interference_neural_networks, mean_time_for_keypoint_extraction, mean_time_for_servoing_loop)*1.4)
ax1.title.set_text('Time Values while Servoing')
ax1.title.set_fontsize(18)
ax1.grid()

# Plot the mean values on the right subplot (ax2)
ax2.axhline(y=mean_interference_neural_networks, color="blue",label="Mean Interference Neural Network")
ax2.axhline(y=mean_time_for_keypoint_extraction, color="orange",label="Mean Time for Interference + Keypoint Extraction")
ax2.axhline(y=mean_time_for_servoing_loop, color="green",label="Mean for Servoing Loop")
ax2.axhline(y=mean_time_for_picture, color="red",label="Mean for Taking a Picture")
ax2.set_ylabel("Time in s", fontsize=14)
legend = ax2.legend(loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2, fontsize=14)
ax2.set_ylim(0, max(mean_interference_neural_networks, mean_time_for_keypoint_extraction, mean_time_for_servoing_loop)*1.4)
ax2.title.set_text('Mean Time Values')
ax2.title.set_fontsize(18)
ax2.grid()

# Display the plot
plt.suptitle('Time Values while Servoing', fontsize=24)
plt.show()

# Get the Progression of the Keypoints while Servoing

In [ ]:
goal_featurepoints = goal_informations[0]
goal_featurepoints = eval(goal_featurepoints)

In [ ]:
# make a list with 64 lists in it
pixel_error = []
for i in range(len(goal_featurepoints)):
    pixel_error.append([])

In [ ]:
for row in servoing_informations:
    featurepoints = row[0]
    featurepoints = eval(featurepoints)

    for i in range(len(featurepoints)):
        pixel_error_x = goal_featurepoints[i][0]-featurepoints[i][0]
        pixel_error_y = goal_featurepoints[i][1]-featurepoints[i][1]
        pixel_error[i].append([pixel_error_x, pixel_error_y])

In [ ]:
keypoint_number = 1 # starts with 0

plt.plot(pixel_error[keypoint_number])
plt.xlabel('Servoing Step')
plt.ylabel('Pixel Error')
plt.title('Pixel Error of Keypoint ' + str(keypoint_number))
plt.legend(['x-error of Keypoint '+str(keypoint_number), 'y-error of the Keypoint '+str(keypoint_number)], loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2)
plt.grid()
plt.show()



In [ ]:
num_keypoints = len(goal_featurepoints)

plt.figure(figsize=(10, 6))  # Optional: Set the size of the figure

for keypoint_number in range(num_keypoints):  # Assuming num_keypoints is the total number of keypoints
    plt.plot(pixel_error[keypoint_number])

plt.xlabel('Iterationes')
plt.ylabel('Pixel Error')
plt.title('Pixel Error of All Keypoints')

# Generate labels for the legend
legend_labels = ['Keypoint {}'.format(keypoint_number) for keypoint_number in range(num_keypoints)]

#plt.legend(legend_labels, loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2)
plt.grid()
plt.show()


# Plot the Velocity of the Endeffector while Servoing

In [ ]:
velocity = [] # hole servoing loop

for i in range(len(servoing_informations)):
    string = servoing_informations[i][5]
    clean_string = re.sub(r'[^\d.eE+-]+', ' ', string)
    values = re.findall(r"[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?", clean_string)
    float_values = [float(value) for value in values]
    velocity.append(float_values)

velocity = list(zip(*velocity))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

ax1.grid()
ax1.plot(velocity[0])
ax1.set_title("Velocity in x-direction")
ax1.set_xlabel("Servoing Step", fontsize=16)
ax1.set_ylabel("Velocity in m/s", fontsize=16)

ax2.grid()
ax2.plot(velocity[1])
ax2.set_title("Velocity in y-direction")
ax2.set_xlabel("Servoing Step", fontsize=16)
ax2.set_ylabel("Velocity in m/s", fontsize=16)

plt.suptitle('Twist while Servoing', fontsize=24)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 2, figsize=(30, 20))
ax1[0].plot(velocity[0])
ax1[0].set_title("Velocity in x-direction")
ax1[0].set_xlabel("Servoing Step")
ax1[0].set_ylabel("Velocity in m/s")
ax1[0].legend()
ax1[0].grid()  

# Plot 2: Velocity in y-direction
ax1[1].plot(velocity[1])
ax1[1].set_title("Velocity in y-direction")
ax1[1].set_xlabel("Servoing Step")
ax1[1].set_ylabel("Velocity in m/s")
ax1[1].legend()
ax1[1].grid()

# Plot 3: Polynomial interpolation for Velocity in x-direction
x = np.arange(len(velocity[0]))
fit_coeffs_x = np.polyfit(x, velocity[0], deg=50)  # Adjust degree as needed
fit_x = np.polyval(fit_coeffs_x, x)
ax2[0].plot(x, velocity[0], label="Data")
ax2[0].plot(x, fit_x, label="Polynomial Fit")
ax2[0].set_title("Polynomial Interpolation for Velocity in x-direction")
ax2[0].set_xlabel("Servoing Step")
ax2[0].set_ylabel("Velocity in m/s")
ax2[0].legend()
ax2[0].grid()

# Plot 4: Polynomial interpolation for Velocity in y-direction
fit_coeffs_y = np.polyfit(x, velocity[1], deg=50)  # Adjust degree as needed
fit_y = np.polyval(fit_coeffs_y, x)
ax2[1].plot(x, velocity[1], label="Data")
ax2[1].plot(x, fit_y, label="Polynomial Fit")
ax2[1].set_title("Polynomial Interpolation for Velocity in y-direction")
ax2[1].set_xlabel("Servoing Step")
ax2[1].set_ylabel("Velocity in m/s")
ax2[1].legend()
ax2[1].grid()

plt.suptitle('Twist while Servoing', fontsize=24)
plt.tight_layout()
plt.show()



# Linarty of Key Points over time


In [ ]:
# We want to analyse the servoing informations[6]
lin_factor = []
for i in range(len(servoing_informations)):
    lin_factor.append(servoing_informations[i][7])
False
# Plot the linear factor
plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(lin_factor)
plt.title('Linear factor')
plt.xlabel('Iteration')
plt.ylabel('Linear factor')
plt.show()
    

# Visualize Robotpose and Robotorientation

In [ ]:
goal_pose = goal_informations[5]
goal_pose = eval(goal_pose)
roll_goal,pitch_goal,yaw_goal = R.from_quat([goal_pose[3],goal_pose[4],goal_pose[5],goal_pose[6]]).as_euler('xyz', degrees=True)

current_pose = []

for i in range(0, len(servoing_informations)):
    servoing_informations[i][8] = eval(servoing_informations[i][8])
    current_pose.append(servoing_informations[i][8])

delta_x = []
delta_y = []
rotation_z = []

for i in range(0, len(servoing_informations)):
    delta_x.append(servoing_informations[i][8][0] - goal_pose[0])
    delta_y.append(servoing_informations[i][8][1] - goal_pose[1])
    roll_current,pitch_current,yaw_current = R.from_quat([servoing_informations[i][8][3],servoing_informations[i][8][4],servoing_informations[i][8][5],servoing_informations[i][8][6]]).as_euler('xyz', degrees=True)
    rotation_z.append(yaw_current - yaw_goal)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

ax1.grid()
ax1.plot(delta_x)
ax1.set_title("Distance to goal in x-direction")
ax1.axhline(0, color='red', linestyle='--') 
ax1.set_xlabel("Servoing Step", fontsize=16)
ax1.set_ylabel("Distance in m", fontsize=16)

ax2.grid()
ax2.plot(delta_y)
ax2.set_title("Distance to goal y-direction")
ax2.axhline(0, color='red', linestyle='--') 
ax2.set_xlabel("Servoing Step", fontsize=16)
ax2.set_ylabel("Distance in m", fontsize=16)

plt.suptitle('Distance to goal while Servoing', fontsize=24)
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(rotation_z)
plt.grid()
plt.title('Angle to goal around z-axis')    
plt.xlabel('Iteration')
plt.ylabel('Angle in degrees')
plt.show()

# Zielbild und Enpose Visueller Vergleich

In [ ]:
import cv2
import numpy as np
from PIL import ImageOps
from PIL import Image, ImageOps

## Pixelerror between the goal Feature Points and the end feature Points

In [ ]:
# print(goal_featurepoints)
# print(servoing_informations[-1][0])
goal_featurepoints = np.array(goal_featurepoints, dtype=np.int64)
end_featurepoints = eval(servoing_informations[-1][0])

In [ ]:
# Berechnung der Differenzen zwischen den Koordinaten in x- und y-Richtung
differences = np.array(goal_featurepoints) - np.array(end_featurepoints)

# Durchschnittliche Pixelfehler in x- und y-Richtung
average_pixel_error = np.mean(abs(differences), axis=0)

print("Durchschnittlicher Pixelfehler in x-Richtung:", average_pixel_error[0])
print("Durchschnittlicher Pixelfehler in y-Richtung:", average_pixel_error[1])


In [ ]:
pixel_in_mm = 1.12 # wieviel enspricht ein Pixel in mm bei Kameraeinstellung im Mittel 870 pixel pro Meter

print("Durchschnittlicher Abweichug in mm in x-Richtung:", average_pixel_error[0]*pixel_in_mm)
print("Durchschnittlicher Abweichug in mm  in y-Richtung:", average_pixel_error[1]*pixel_in_mm) 

## Vizualation of the End and Goal Feature Points in the goal Image

In [ ]:
def img_patch(img, im_size):
    # Calculate the aspect ratio of the image
    img_ratio = img.width / img.height

    # If the image is wider than it is high
    if img_ratio > 1:
        # Scale the width to the desired size and calculate the height while maintaining the aspect ratio
        new_width = im_size[0]
        new_height = int(new_width / img_ratio)
    # If the image is taller than it is wide
    else:
        # Scale the height to the desired size and calculate the width while maintaining the aspect ratio
        new_height = im_size[1]
        new_width = int(new_height * img_ratio)

    # Resize the image to the calculated size while maintaining the aspect ratio
    img = img.resize((new_width, new_height), resample=Image.BICUBIC)

    # Add black borders to the left and right or top and bottom of the image to make it the desired size
    delta_w = im_size[0] - new_width
    delta_h = im_size[1] - new_height
    pad_width = delta_w // 2
    pad_height = delta_h // 2
    padding = (pad_width, pad_height, delta_w - pad_width, delta_h - pad_height)
    
    img = ImageOps.expand(img, border=padding, fill=(255,255,255))

    # Return the scaled and centered image
    return img



### Goal image

In [ ]:
# Load the image
img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/goal_pic/goal_picture.png')

# Define the desired size
desired_size = (720, 720)

# Call the img_patch function
patched_img = img_patch(img, desired_size)


In [ ]:
# Add the Keypoints of the Goal Image
# Convert the PIL Image to a numpy array (and convert RGB to BGR for use with cv2)
patched_img = cv2.cvtColor(np.array(patched_img), cv2.COLOR_RGB2BGR)

scaled_featurepoints = []
for point in goal_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)

    # Färbe den Pixel rot
    patched_img[scaled_point[1], scaled_point[0]] = [0, 0, 255]  # BGR color

cv2.imwrite('servoing_csv/validation_picture/validation_only_goal_picture.png', patched_img)

# zeige das Bild an
plt.imshow(patched_img)


### Current Image

In [ ]:
# Load the image
img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/current_pic/current_picture.png')

# Define the desired size
desired_size = (720, 720)

# Call the img_patch function
patched_img = img_patch(img, desired_size)

In [ ]:
patched_img = cv2.cvtColor(np.array(patched_img), cv2.COLOR_RGB2BGR)

scaled_featurepoints = []
for point in end_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)
    #Färbe den Pixel
    patched_img[scaled_point[1], scaled_point[0]] = [255, 0, 0]  # Blue

cv2.imwrite('servoing_csv/validation_picture/validation_only_current_picture.png', patched_img)

# zeige das Bild an
plt.imshow(patched_img)

### Both, based current

In [ ]:
# Load the image
#img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/goal_pic/goal_picture.png')
img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/current_pic/current_picture.png')

# Define the desired size
desired_size = (720, 720)

# Call the img_patch function
patched_img = img_patch(img, desired_size)


In [ ]:
# Add the Keypoints of the Goal Image
# Convert the PIL Image to a numpy array (and convert RGB to BGR for use with cv2)
patched_img = cv2.cvtColor(np.array(patched_img), cv2.COLOR_RGB2BGR)

scaled_featurepoints = []
for point in goal_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)

    # Färbe den Pixel rot
    patched_img[scaled_point[1], scaled_point[0]] = [0, 0, 255]  # BGR color

# Add the Keypoints of the end Image
# Coloring for servoing_information points in blue
scaled_featurepoints = []
for point in end_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)
    #Färbe den Pixel
    patched_img[scaled_point[1], scaled_point[0]] = [255, 0, 0]  # Blue
#Save the edited image
cv2.imwrite('servoing_csv/validation_picture/validation_both_based_current_picture.png', patched_img)

# zeige das Bild an
plt.imshow(patched_img)

# zeige das Bild an
plt.imshow(patched_img)

### Both, based goal


In [ ]:
# Load the image
img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/goal_pic/goal_picture.png')
#img = Image.open('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/current_pic/current_picture.png')

# Define the desired size
desired_size = (720, 720)

# Call the img_patch function
patched_img = img_patch(img, desired_size)


In [ ]:
# Add the Keypoints of the Goal Image
# Convert the PIL Image to a numpy array (and convert RGB to BGR for use with cv2)
patched_img = cv2.cvtColor(np.array(patched_img), cv2.COLOR_RGB2BGR)

scaled_featurepoints = []
for point in goal_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)

    # Färbe den Pixel rot
    patched_img[scaled_point[1], scaled_point[0]] = [0, 0, 255]  # BGR color

# Add the Keypoints of the end Image
# Coloring for servoing_information points in blue
scaled_featurepoints = []
for point in end_featurepoints:
    # Multipliziere jeden Punkt mit dem Faktor 2
    scaled_point = [coordinate * 2 for coordinate in point]
    scaled_featurepoints.append(scaled_point)
    #Färbe den Pixel
    patched_img[scaled_point[1], scaled_point[0]] = [255, 0, 0]  # Blue
#Save the edited image
cv2.imwrite('servoing_csv/validation_picture/validation_both_based_goal_picture.png', patched_img)

# zeige das Bild an
plt.imshow(patched_img)